## Importación de librerías

In [4]:
import pandas as pd

## Carga de datos

In [5]:
df = pd.read_excel('../data/raw/Datos_originales.xlsx')

## Selección de columnas

Vamos a eliminar columnas que no nos aporten información relevante.

- Descripcion provincia: Tiene el 100% de nulos, pero sería una columna categórica con el nombre de la provincia según su CP
- Asiento: número de apunte contable, sin relevancia
- Perfíl: Adminitrativo que introduce el dato. Sin relevania
- Opción: Identificador del tipo de cuenta. Sin relevancia
- Provincia: Está todo en 0, podríamos eliminarla




In [ ]:
df = df.drop_duplicates()

In [9]:
columnas_eliminar = ['Descripción Provinc', 'Asiento','Perfíl', 'Opción', 'Provincia']

In [10]:
df = df.drop(columns=columnas_eliminar)

In [14]:
df.duplicated().sum()

np.int64(6)

No hay apenas duplicados, por lo que los dejamos

## Limpieza de datos

Realizar una limpieza columan por columna

In [15]:
df.columns

Index(['SITUACION MES', 'SITUACION AÑO', 'Estrato', 'Linea Venta',
       'Descripción Linea V', 'Subcuenta', 'Descripción Subcuen', 'Concepto',
       'Cliente', 'Cont/Proy', 'Línea', 'Importe', 'I/C', 'Resultado',
       'Responsable'],
      dtype='object')

## Unificamos Fecha

In [16]:
df['Fecha'] = pd.to_datetime(dict(year=df['SITUACION AÑO'],
                                   month=df['SITUACION MES'],
                                   day=1))

## Eliminamos columnas de SITUACION MES Y SITUACION AÑO

In [17]:
df = df.drop(columns=['SITUACION MES', 'SITUACION AÑO'])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60302 entries, 0 to 60301
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Estrato              60302 non-null  int64         
 1   Linea Venta          60302 non-null  int64         
 2   Descripción Linea V  60302 non-null  object        
 3   Subcuenta            60302 non-null  int64         
 4   Descripción Subcuen  57287 non-null  object        
 5   Concepto             60302 non-null  object        
 6   Cliente              59262 non-null  float64       
 7   Cont/Proy            60302 non-null  int64         
 8   Línea                60302 non-null  int64         
 9   Importe              60302 non-null  int64         
 10  I/C                  60302 non-null  int64         
 11  Resultado            60302 non-null  int64         
 12  Responsable          60302 non-null  object        
 13  Fecha                60302 non-

In [28]:
df.isnull().mean() * 100

Estrato                0.000000
Linea Venta            0.000000
Descripción Linea V    0.000000
Subcuenta              0.000000
Descripción Subcuen    4.999834
Concepto               0.000000
Cliente                1.724653
Cont/Proy              0.000000
Línea                  0.000000
Importe                0.000000
I/C                    0.000000
Resultado              0.000000
Responsable            0.000000
Fecha                  0.000000
dtype: float64

In [24]:
df.sample(5)

,Estrato,Linea Venta,Descripción Linea V,Subcuenta,Descripción Subcuen,Concepto,Cliente,Cont/Proy,Línea,Importe,I/C,Resultado,Responsable,Fecha
438,35,14,SERV. AUXILIARES,3,PRODUCCION VALORADA SPYC,PRODUCCION REALIZADA 01/2023,296890.0,2,4086,4252,7,4252,Fernando_Trujillo,2023-01-01
54796,35,14,SERV. AUXILIARES,617,CARGAS SOCIALES,AJUSTE 11/2023 T90 SILTRA,823430.0,3,43946,1,3,-1,Fernando_Trujillo,2023-12-01
25724,2,14,SERV. AUXILIARES,666,SERV. AUX. TERCEROS,PERIOD. LICENCIA GOOGLE 2023,766090.0,1649,41,445,3,-445,Mercedes_Hernandez,2023-06-01
7861,28,82,SERV. ESCOLARES,617,CARGAS SOCIALES,AJUSTE 01/2023 T90 SILTRA,911530.0,6,31084,1,3,-1,Mercedes_Hernandez,2023-02-01
9509,35,14,SERV. AUXILIARES,3,PRODUCCION VALORADA SPYC,PRODUCCION REALIZADA 03/2023,548620.0,1,4161,26895,7,26895,Fernando_Trujillo,2023-03-01


## Cosas a limpiar

- 'Estrato' -> Es el código postal, esta en valor númeríco pero podemos pasarlo a categórico
- 'Linea de venta' -> Es el identificador de la linea de negocio, podemos pasarlo a categórico
- 'Subcuenta' -> Cuenta contable, pasarlo a númerico
- 'Descripción Subcuent' -> Descripcion  de la cuenta contable, tiene nulos, 5%
- 'Cliente' -> Identificador del cliente, es categórica. Tiene un 1,72% de nulos.
- 'Cont/Proy -> Identificdor del contrato, es categórica


In [30]:
df.select_dtypes(include='object').columns

Index(['Descripción Linea V', 'Descripción Subcuen', 'Concepto',
       'Responsable'],
      dtype='object')

In [31]:
for col in df.select_dtypes(include='object').columns:
    print(col.upper())
    print(df[col].value_counts())
    print('------')


DESCRIPCIÓN LINEA V
Descripción Linea V
SERV. AUXILIARES    35368
SERV. LOGÍSTICOS     6786
SERV. DEPORTIVOS     4276
FACILITY S.AUXIL     3316
SERV. ESCOLARES      3100
SERV.COMERCIALES     2565
BPO                  2509
ARTE Y CULTURA       1590
OCIO/TIEMPO LIBR      665
S.AUX.HOTEL/CRUC      111
TELEMARKETING          16
Name: count, dtype: int64
------
DESCRIPCIÓN SUBCUEN
Descripción Subcuen
CARGAS SOCIALES             18120
SUELDOS Y SALARIOS           9955
AMORT. INMOV. MATERIAL       5772
SERV. AUX. TERCEROS          4592
PRODUCCION VALORADA SPYC     3637
MATERIALES                   2948
COMUNICACIONES               1969
ALQUILERES                   1684
TRABAJOS SUBCONTRATADOS      1593
SERV. PENDIENTES FACTUR.     1377
UNIFORMES                     890
OTROS GTOS.DE PERSONAL        811
SERV. AUX. EMPR. GRUPO        634
HORAS EXTRA                   571
GASTOS DE VIAJE               562
REP.,CONSV.Y OTROS GASTO      381
DIETAS                        371
FACTURAC.SIST.ANTERIOR 